In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc

from sklearn import tree
import graphviz 

from collections import Counter

import copy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import calendar
import datetime
from datetime import datetime
%matplotlib inline

/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
car_df= pd.read_csv('newCardata.csv')

In [3]:
#replace map
replace_Month = {'Month':{'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
                          'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}}

replace_MonthClaimed = {'MonthClaimed':{'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
                          'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}}

car_df.replace(replace_Month,inplace=True)
car_df.replace(replace_MonthClaimed,inplace=True)

car_df.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy
0,12,5,Wednesday,Honda,Urban,Tuesday,1,1,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,1,3,Wednesday,Honda,Urban,Monday,1,4,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,10,5,Friday,Honda,Urban,Thursday,11,2,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,6,2,Saturday,Toyota,Rural,Friday,7,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,1,5,Monday,Honda,Urban,Tuesday,2,2,Female,Single,...,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


In [4]:
#function to calculate the no of days passed between the accident and the claims.
# Reporting Gap:

def get_day(year,month,weekOfMonth,dayOfWeek):
    count = 0
    c = calendar.TextCalendar(calendar.SUNDAY)
    l = []
    for i in c.itermonthdates(year,month):
        l.append(i)
    for j in range(len(l)):
        day = calendar.day_name[l[j].weekday()]
        if day == dayOfWeek:
            count += 1
        if count == weekOfMonth:
            return l[j]
            break

def differ_days(date1,date2):
    a = date1
    b = date2
    return (a - b).days


In [5]:
day_diff = np.zeros((car_df.shape[0],1))
i = 0
for i in range(car_df.shape[0]):
    
    if(car_df['MonthClaimed'][i]-car_df['Month'][i]) < 0:
        year2 = car_df['Year'][i] + 1
        month2 = car_df['MonthClaimed'][i]
        week2 = car_df['WeekOfMonthClaimed'][i]
        day2 = car_df['DayOfWeekClaimed'][i]
        year1 = car_df['Year'][i]
        month1 = car_df['Month'][i]
        week1 = car_df['WeekOfMonth'][i]
        day1 = car_df['DayOfWeek'][i]
        day_diff[i] = differ_days(get_day(year2,month2,week2,day2),get_day(year1,month1,week1,day1))
    else:
        year2 = car_df['Year'][i]
        month2 = car_df['MonthClaimed'][i]
        week2 = car_df['WeekOfMonthClaimed'][i]
        day2 = car_df['DayOfWeekClaimed'][i]
        year1 = car_df['Year'][i]
        month1 = car_df['Month'][i]
        week1 = car_df['WeekOfMonth'][i]
        day1 = car_df['DayOfWeek'][i]
        day_diff[i] = differ_days(get_day(year2,month2,week2,day2),get_day(year1,month1,week1,day1))
        

In [6]:
#adding column to the existing dataframe
car_df['daysDiff'] = day_diff 
car_df.head(3)


,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy,daysDiff
0,12,5,Wednesday,Honda,Urban,Tuesday,1,1,Female,Single,...,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability,6.0
1,1,3,Wednesday,Honda,Urban,Monday,1,4,Male,Single,...,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision,5.0
2,10,5,Friday,Honda,Urban,Thursday,11,2,Male,Married,...,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision,13.0


In [7]:
#now drop the original attibutes, like 'Month' column(we don't need anymore)
car_df.drop(['Month'],axis=1,inplace=True)
car_df.drop(['MonthClaimed'],axis=1,inplace=True)
car_df.drop(['DayOfWeek'],axis=1,inplace=True)
car_df.drop(['DayOfWeekClaimed'],axis=1,inplace=True)
car_df.drop(['WeekOfMonth'],inplace=True,axis=1)
car_df.drop(['WeekOfMonthClaimed'],inplace=True,axis=1)
car_df.drop(['VehicleCategory'],axis=1,inplace=True)
car_df.drop(['PolicyType'],axis=1,inplace=True)
car_df.drop(['AgeOfPolicyHolder'],inplace=True,axis=1)
car_df.drop(['Year'],inplace=True,axis=1)
car_df.drop(['RepNumber'],axis=1,inplace=True)
car_df.drop(['PolicyNumber'],inplace=True,axis=1)

In [8]:
car_df= car_df.replace(0,17)
car_df.head()

,Make,AccidentArea,Sex,MaritalStatus,Age,Fault,VehiclePrice,FraudFound,Deductible,DriverRating,...,PastNumberOfClaims,AgeOfVehicle,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,BasePolicy,daysDiff
0,Honda,Urban,Female,Single,21,Policy Holder,"more than 69,000",No,300,1,...,none,3 years,No,No,External,none,1 year,3 to 4,Liability,6.0
1,Honda,Urban,Male,Single,34,Policy Holder,"more than 69,000",No,400,4,...,none,6 years,Yes,No,External,none,no change,1 vehicle,Collision,5.0
2,Honda,Urban,Male,Married,47,Policy Holder,"more than 69,000",No,400,3,...,1,7 years,No,No,External,none,no change,1 vehicle,Collision,13.0
3,Toyota,Rural,Male,Married,65,Third Party,"20,000 to 29,000",No,400,2,...,1,more than 7,Yes,No,External,more than 5,no change,1 vehicle,Liability,20.0
4,Honda,Urban,Female,Single,27,Third Party,"more than 69,000",No,400,1,...,none,5 years,No,No,External,none,no change,1 vehicle,Collision,15.0


In [9]:
#get the lable of the datasets


label_Number = LabelEncoder()  #object of lable encoder

#conver label to Number
car_df['FraudFound'] = label_Number.fit_transform(car_df['FraudFound'].astype('str'))

yLable = car_df['FraudFound']

#drop the lable from the dataset
car_df.drop(['FraudFound'],inplace=True,axis=1)


In [10]:
car_df.head()



,Make,AccidentArea,Sex,MaritalStatus,Age,Fault,VehiclePrice,Deductible,DriverRating,Days:Policy-Accident,...,PastNumberOfClaims,AgeOfVehicle,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,BasePolicy,daysDiff
0,Honda,Urban,Female,Single,21,Policy Holder,"more than 69,000",300,1,more than 30,...,none,3 years,No,No,External,none,1 year,3 to 4,Liability,6.0
1,Honda,Urban,Male,Single,34,Policy Holder,"more than 69,000",400,4,more than 30,...,none,6 years,Yes,No,External,none,no change,1 vehicle,Collision,5.0
2,Honda,Urban,Male,Married,47,Policy Holder,"more than 69,000",400,3,more than 30,...,1,7 years,No,No,External,none,no change,1 vehicle,Collision,13.0
3,Toyota,Rural,Male,Married,65,Third Party,"20,000 to 29,000",400,2,more than 30,...,1,more than 7,Yes,No,External,more than 5,no change,1 vehicle,Liability,20.0
4,Honda,Urban,Female,Single,27,Third Party,"more than 69,000",400,1,more than 30,...,none,5 years,No,No,External,none,no change,1 vehicle,Collision,15.0


In [11]:
#replace categorical by Number
#replace map
replace_Make = {'Make':{'Pontiac':1,'Toyota':2,'Honda':3,'Mazda':4,'Chevrolet':5,'Accura':6,
                          'Ford':7,'VW':8,'Dodge':9,'Saab':10,'Mercury':11,'Saturn':12,
                       'Nisson':13,'BMW':14,'Jaguar':15,'Porche':16,'Mecedes':17,'Ferrari':18,'Lexus':19}}

replace_AccidentArea = {'AccidentArea':{'Urban':1,'Rural':2}}

replace_Sex = {'Sex':{'Male':1,'Female':2}}

replace_MaritalStatus = {'MaritalStatus':{'Married':1,'Single':2,'Divorced':3,'Widow':4}}

replace_VehiclePrice = {'VehiclePrice':{'20,000 to 29,000':1,'30,000 to 39,000':2,'more than 69,000':3,
                                        'less than 20,000':4,'40,000 to 59,000':5,'60,000 to 69,000':6}}

replace_Fault = {'Fault':{'Policy Holder':1,'Third Party':2}}

replace_VehicleCategory = {'VehicleCategory':{'Sedan':1,'Sport':2,'Utility':3}}

replace_DaysPolicyAccident = {'Days:Policy-Accident':{'more than 30':1,'8 to 15':2,'none':3,'15 to 30':4,
                                                        '1 to 7':5}}
replace_DaysPolicyClaim = {'Days:Policy-Claim':{'more than 30':1,'15 to 30':2,'8 to 15':3,'none':4}}

replace_PastNumberOfClaims = {'PastNumberOfClaims':{'2 to 4':1,'none':2,'1':3,'more than 4':4}}

replace_AgeOfVehicle = {'AgeOfVehicle':{'7 years':1,'more than 7':2,'6 years':3,'5 years':4,
                                        'new':5,'4 years':6,'3 years':7,'2 years':8}}

replace_PoliceReportFiled = {'PoliceReportFiled':{'No':1,'Yes':2}}

replace_WitnessPresent = {'WitnessPresent':{'No':1,'Yes':2}}

replace_AgentType = {'AgentType':{'External':1,'Internal':2}}

replace_NumberOfSuppliments = {'NumberOfSuppliments':{'none':1,'more than 5':2,'1 to 2':3,'3 to 5':4}}

replace_AddressChangeClaim = {'AddressChange-Claim':{'no change':1,'4 to 8 years':2,'2 to 3 years':3,
                                                     '1 year':4,'under 6 months':5}}

replace_NumberOfCars = {'NumberOfCars':{'1 vehicle':1,'2 vehicles':2,'3 to 4':3,
                                                     '5 to 8':4,'more than 8':5}}

replace_BasePolicy = {'BasePolicy':{'Collision':1,'Liability':2,'All Perils':3}}

In [12]:
car_df.replace(replace_Make,inplace=True)

car_df.replace(replace_AccidentArea,inplace=True)

car_df.replace(replace_Sex,inplace=True)

car_df.replace(replace_MaritalStatus,inplace=True)

car_df.replace(replace_VehiclePrice,inplace=True)

car_df.replace(replace_VehicleCategory,inplace=True)

car_df.replace(replace_DaysPolicyAccident,inplace=True)

car_df.replace(replace_DaysPolicyClaim,inplace=True)

car_df.replace(replace_PastNumberOfClaims,inplace=True)

car_df.replace(replace_AgeOfVehicle,inplace=True)

car_df.replace(replace_PoliceReportFiled,inplace=True)

car_df.replace(replace_WitnessPresent,inplace=True)

car_df.replace(replace_AgentType,inplace=True)

car_df.replace(replace_NumberOfSuppliments,inplace=True)

car_df.replace(replace_AddressChangeClaim,inplace=True)

car_df.replace(replace_NumberOfCars,inplace=True)

car_df.replace(replace_BasePolicy,inplace=True)

car_df.replace(replace_Fault,inplace=True)



In [13]:
car_df.head()

,Make,AccidentArea,Sex,MaritalStatus,Age,Fault,VehiclePrice,Deductible,DriverRating,Days:Policy-Accident,...,PastNumberOfClaims,AgeOfVehicle,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,BasePolicy,daysDiff
0,3,1,2,2,21,1,3,300,1,1,...,2,7,1,1,1,1,4,3,2,6.0
1,3,1,1,2,34,1,3,400,4,1,...,2,3,2,1,1,1,1,1,1,5.0
2,3,1,1,1,47,1,3,400,3,1,...,3,1,1,1,1,1,1,1,1,13.0
3,2,2,1,1,65,2,1,400,2,1,...,3,2,2,1,1,2,1,1,2,20.0
4,3,1,2,2,27,2,3,400,1,1,...,2,4,1,1,1,1,1,1,1,15.0


In [14]:
#train the model
X_train,X_test,y_train,y_test = train_test_split(car_df,yLable,random_state=3,test_size=0.25)

In [15]:
#initialize the model
classification_tree= tree.DecisionTreeClassifier()

In [16]:
featureName=list(X_train.columns.values)
lableName=['Fraud','Non-Fraud']

In [17]:
classification_tree= classification_tree.fit(X_train,y_train)

In [18]:
dot_data = tree.export_graphviz(classification_tree, out_file=None, 
                     feature_names=featureName,  
                     class_names=lableName,  
                     filled=True, rounded=True,  
                     special_characters=True)

In [54]:
graph = graphviz.Source(dot_data)  
graph.render("carClaimsXG")

'carClaimsXG.pdf'

In [19]:
print('Decision classifier:')
predicted = classification_tree.predict(X_test)
print(type(predicted),predicted.shape,predicted)
print('Accuracy is ',round(accuracy_score(y_test,classification_tree.predict(X_test)) * 100,2))


Decision classifier:
<class 'numpy.ndarray'> (3855,) [0 0 0 ..., 0 0 0]
Accuracy is  88.77


In [20]:
# calculating specifity and sensitivity
# 0  := Negative
# 1 := Positive
cm = confusion_matrix(y_test,predicted)
print("Confusion Matrix:\n",cm)

TN, FP, FN, TP = cm.ravel()
print("TN:",TN)
print("FP:",FP)
print("FN:",FN)
print("TP:",TP)

print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)


Confusion Matrix:
 [[3380  229]
 [ 204   42]]
TN: 3380
FP: 229
FN: 204
TP: 42
Accuracy: 88.7678339818
Sensitivity: 17.0731707317
Specificity: 93.6547520089


In [21]:
#apply SMOTE
from imblearn.over_sampling import SMOTE 
sm = SMOTE()
features,labels = sm.fit_sample(car_df,yLable)
print(features.shape,labels.shape)
X_train,X_test,y_train,y_test = train_test_split(features,labels,random_state=3,test_size=0.25)


(28992, 21) (28992,)


In [22]:
#initialize the model
classification_tree= tree.DecisionTreeClassifier()
classification_tree= classification_tree.fit(X_train,y_train)

print('Decision classifier:')
predicted = classification_tree.predict(X_test)
print(type(predicted),predicted.shape,predicted)
print('Accuracy is ',round(accuracy_score(y_test,classification_tree.predict(X_test)) * 100,2))

# calculating specifity and sensitivity
# 0  := Negative
# 1 := Positive
cm = confusion_matrix(y_test,predicted)
print("Confusion Matrix:\n",cm)

TN, FP, FN, TP = cm.ravel()
print("TN:",TN)
print("FP:",FP)
print("FN:",FN)
print("TP:",TP)

print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)



Decision classifier:
<class 'numpy.ndarray'> (7248,) [1 1 0 ..., 0 0 1]
Accuracy is  94.4
Confusion Matrix:
 [[3419  204]
 [ 202 3423]]
TN: 3419
FP: 204
FN: 202
TP: 3423
Accuracy: 94.3984547461
Sensitivity: 94.4275862069
Specificity: 94.369307204


In [24]:
#cross validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn import model_selection
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = model_selection.KFold(n_splits=10, random_state=10)
model= tree.DecisionTreeClassifier(random_state= 42)

results = model_selection.cross_validate(estimator=model,X=features,y=labels,cv=kfold,scoring=scoring)

print(np.mean(results['test_accuracy']))
print(np.mean(results['test_precision']))
print(np.mean(results['test_recall']))
print(np.mean(results['test_f1_score']))


0.942504942251
0.597929900074
0.622588228252
0.608427477743
